<a href="https://colab.research.google.com/github/RohanSaxena14/snapchat_filters/blob/master/snapchat_filters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Collecting all the relevant data** 

In [0]:
!wget "https://github.com/italojs/facial-landmarks-recognition/raw/master/shape_predictor_68_face_landmarks.dat"
!wget "http://arunponnusamy.com/files/mmod_human_face_detector.dat"
!wget "https://github.com/RohanSaxena14/snapchat_filters/raw/master/data/mr bean.mp4"
!wget "https://github.com/RohanSaxena14/snapchat_filters/raw/master/data/Eye.png"
!wget "https://github.com/RohanSaxena14/snapchat_filters/raw/master/data/tongue.png"

# **Importing all relevant packages and initializing the dlib's face detector and face_landmark detector**

In [0]:
import cv2
import dlib
import numpy as np
from google.colab.patches import cv2_imshow


cnn_face_detector = dlib.cnn_face_detection_model_v1("/content/mmod_human_face_detector.dat") # The cnn face detector in dlib
face_detector = dlib.get_frontal_face_detector() # The other face detector in dlib
predictor = dlib.shape_predictor("/content/shape_predictor_68_face_landmarks.dat") #The facial_keypoint detector in dlib

# **This functions inserts the clipart in our frame and takes care of its shape as well**
# i have used the concept of fore_ground and back_ground here to combine our clipart with the frame.

In [0]:
# This function takes in the frame, the size and points and out clipart and combine the image

def insert_clipart(frame, points, size, clipart):
  clipart = cv2.resize(clipart, (2*size, 2*size)) # Resizing the clipart into appropriate size
  orig_mask = clipart[:,:,3] # The 4th layer in our clipart image is the transparency layer, we are using it as a mask to cut-out the exact shape ou the eye
  inv_mask = 255 - orig_mask # The inverse mask will be used to mask out the parts from out frame
  clipart = clipart[:, :, 0:3] # After getting the mask, we are converting the image back to a normal BGR image

  cut = frame[points[1]: points[1] + 2*size, points[0]: points[0] + 2*size, :] # The relevant part from the image is cut for manipulation
  
  # Using mask approprite cuts are made

  cut = cv2.bitwise_and(cut, cut, mask = inv_mask) 
  clipart = cv2.bitwise_and(clipart, clipart, mask = orig_mask)

  cut = cv2.add(cut, clipart) # Adding the two cuts

  frame[points[1]:points[1]+ 2*size, points[0]:points[0]+ 2*size, :] = cut # Returning the cut to the main frame after processing

  return frame

# **Mainly we are concerned with the points around eyes here, the main code is here, i have added appropriate comments for better understanding**

In [0]:
cap = cv2.VideoCapture("mr bean.mp4")  # The video we are doing editing on

ret, image = cap.read()

fourcc = cv2.VideoWriter_fourcc(*'XVID')

out = cv2.VideoWriter('output.avi',fourcc, 20.0, (image.shape[1], image.shape[0])) # For storing processed frames

eye = cv2.imread("/content/Eye.png", -1) # Our eye image, note that we are loading the image as-it-is as it has transparency properties

while cap.isOpened():  # Looping on frames on our video
    ret, image = cap.read()

    if ret == False:
      break
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = cnn_face_detector(image, 1) # Detecting faces using dlib

    for face in faces: # Looping on all detected faces
      
      # This code draws rectangle around faces

      '''

      x = face.rect.left()
      y = face.rect.top()
      w = face.rect.right() - x
      h = face.rect.bottom() - y
      
      cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 3)
      '''
      

      points = predictor(gray, face.rect) # Predicting all 68 points on the detected face

      
      for i in range(0, 68): # Looping around all he points and plotting them on our frame
        point = (points.part(i).x, points.part(i).y)

        cv2.circle(image, point, 1, (0, 255, 0), -1)
      
      # Taking all the relevent points

      p_38 = np.array([int((points.part(37).x + points.part(38).x)/2), int((points.part(37).y + points.part(38).y)/2)])
      p_41 = np.array([int((points.part(40).x + points.part(41).x)/2), int((points.part(40).y + points.part(41).y)/2)])

      p_44 = np.array([int((points.part(43).x + points.part(44).x)/2), int((points.part(43).y + points.part(44).y)/2)])
      p_47 = np.array([int((points.part(46).x + points.part(47).x)/2), int((points.part(46).y + points.part(47).y)/2)])

      p_43 = np.array([points.part(42).x, points.part(42).y])
      p_46 = np.array([points.part(45).x, points.part(45).y])

      p_37 = np.array([points.part(36).x, points.part(36).y])
      p_40 = np.array([points.part(39).x, points.part(39).y])

      # Calculating distances is very beneficial in deciding both the position and size of our clipart  

      eye_vertical_left = np.linalg.norm(p_38 - p_41).astype(int)
      eye_vertical_right = np.linalg.norm(p_44 - p_47).astype(int)

      eye_horizontal_right = np.linalg.norm(p_43 - p_46).astype(int)
      eye_horizontal_left = np.linalg.norm(p_37 - p_40).astype(int)

      #if eye_vertical_left/eye_horizontal_right >= 0.4 and eye_vertical_right/eye_horizontal_right >= 0.4: #if you want to add eyes only when your eyes are wide open
      if True: #when you want to add eyes on all frames
        corner_x = p_37[0] - int(eye_horizontal_left / 2)
        corner_y = p_37[1] - eye_horizontal_left

        points_left_eye = (corner_x, corner_y)

        image = insert_clipart(image, points_left_eye, eye_horizontal_left, eye)

        corner_x = p_43[0] - int(eye_horizontal_right / 2)
        corner_y = p_43[1] - eye_horizontal_right

        points_right_eye = (corner_x, corner_y)

        # Calling the function we made earlier

        image = insert_clipart(image, points_right_eye, eye_horizontal_right, eye)  
    

    out.write(image) # Writing the processed frames

cap.release()
out.release()

# **HAPPY LEARNING !!!!! 😄**